In [1]:
from tkinter import ttk
from tkinter import *
from tkinter import scrolledtext as st
import pandas as pd
import json
import requests
import urllib3.contrib.pyopenssl
from requests.adapters import HTTPAdapter
import warnings
import time
warnings.filterwarnings('ignore', category=DeprecationWarning)
urllib3.contrib.pyopenssl.inject_into_urllib3()
urllib3.disable_warnings()

id_dict = {'HSDB':11933, 'CAMEO':11944}

s = requests.session()
s.mount('http://', HTTPAdapter(max_retries=100000))
s.mount('https://', HTTPAdapter(max_retries=100000))
s.keep_alive = False

root = Tk()
root.title('Query Chem')
root.geometry('860x500')
root.resizable(0, 0)

def get_total_page(url:str):
    
    res = requests.get(url, verify=False, headers={'Connection': 'close'}, timeout=(100000 ,50))
    res_json = json.loads(res.text, strict=False)
    res.close()

    return int(res_json["Annotations"]["TotalPages"])


def get_page_url(heading:str, 
                 type:str, 
                 id:int,
                 page:int):
    url = (f'https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/annotations/heading/JSON/?source={id}&heading_type={type}&heading={heading}&'
          + f'page={page}'
          + '&response_display')
    return url


def get_subdict_value_in_list(list:list, key:str):
    for i in list:
        if i.get(key):
            return i.get(key)


def get_chemname_property(heading:str, 
                 type:str, 
                 id:int):
    
    init_page = 1
    init_url = get_page_url(heading, type, id, init_page)
    cur_total_page = get_total_page(init_url)
    total_name_property = {}

    for i in range(cur_total_page):
        url = get_page_url(heading, type, id, init_page)
        res = requests.get(url, verify=False, headers={'Connection': 'close'}, timeout=(100000 ,50))
        anid_list = res.json()["Annotations"]["Annotation"]

        for i in anid_list:
            value_list = []
            name = i['Name']
            data_list = i["Data"]
            value = get_subdict_value_in_list(data_list, key="Value")
            reference = get_subdict_value_in_list(data_list, key="Reference")
            property_list = value["StringWithMarkup"]
            property = get_subdict_value_in_list(property_list, key='String')

            if name not in total_name_property.keys():
                value_list.append(property)
                if reference:
                    for refer in reference:
                        value_list.append(refer)
                else:
                    value_list.append('None reference provided')
                total_name_property[name] = value_list
        
        res.close()
        init_page += 1

    return total_name_property


def get_property(id):

    url = f'https://pubchem.ncbi.nlm.nih.gov/rest/pug/annotations/sourcename/JSON/?sourcename={id}&response_Type=display'
    req = requests.get(url, verify=False, headers={'Connection': 'close'}, timeout=(100000 ,50))
    proper_json = req.json() # json.loads(str(req.text))
    head_type_list = proper_json["InformationList"]["Annotation"]
    req.close()
    proper_dict = {}

    for i in range(len(head_type_list)):
        heading = head_type_list[i]['Heading']
        type = head_type_list[i]['Type']
        if heading not in proper_dict:
            proper_dict[heading] = type
    
    return proper_dict, proper_dict.keys()


# combobox Labelframe
cbx_group = LabelFrame(root, text="Select", font=('Arial', 13,"bold"))
cbx_group.grid(row=0, column=0, columnspan=2, padx=20, pady=10)

# database label
database_lb = ttk.Label(cbx_group, text='Databese', font=("Arial",10,"bold"))
database_lb.grid(row=0, column=0)

# database select combobox
database_var = StringVar()
database_cbx = ttk.Combobox(cbx_group,
                            font=("Arial",11,"bold"),
                            textvariable=database_var,
                            value=("HSDB", "CAMEO"),
                            width=15,
                            state='readonly')
database_cbx.grid(row= 1, column=0, padx=20, pady=5)


def get_database_id(event):
    id = id_dict[database_var.get()]
    property_cbx['value'] = tuple(get_property(id)[1])


database_cbx.bind('<<ComboboxSelected>>', get_database_id)


#property_label
property_lb = ttk.Label(cbx_group, text='property', font=("Arial",10,"bold"))
property_lb.grid(row=0,column=1)

#property select
property_var = StringVar()
property_cbx = ttk.Combobox(cbx_group,
                            font=("Arial",10,"bold"),
                            textvariable=property_var,
                            width=30,
                            state='readonly')
property_cbx.grid(row= 1, column=1, padx=5, pady=10)

def get_results():
    heading = property_var.get()
    id = id_dict[database_var.get()]
    type = get_property(id)[0][heading]
    print(heading, type)
    total_name_property = get_chemname_property(heading, type, id)
    Entrez_name = input_entry.get().upper()
    if Entrez_name in total_name_property.keys():
        result = total_name_property[Entrez_name]
        refer_st.insert('end', '\n')
        refer_st.insert('end',
                        time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
                        +': '
                        + str(result)
                        +'\n')
    else:
        refer_st.insert('end', '\n')
        refer_st.insert('end',
                        time.strftime('%Y-%m-%d %H:%M:%S', time.localtime())
                        +': '
                        + 'The current database does not contain'
                        + heading + ' ' + 'of '
                        +Entrez_name + ','
                        +' please change the database or check the Entrez name your entered'
                        +'\n')
        
# result labelframe
result_group = LabelFrame(root, text="Result", font=('Arial', 13,"bold"))
result_group.grid(row=2, column=1, columnspan=4, padx=20, pady=10)

#reference
refer_st = st.ScrolledText(result_group,
                           width=110,
                           height=20,
                           relief='solid',
                           font=('Arial', 10))
refer_st.grid(row=2, column=0, columnspan=4, padx=10, pady=10)


# Entrez labelframe
Entrez_group = LabelFrame(root, text="Select", font=('Arial', 13,"bold"))
Entrez_group.grid(row=0, column=2, columnspan=2, padx=0, pady=10)

# input label
input_lb = ttk.Label(Entrez_group, text='Chemical name', font=('Arial', 10,"bold"))
input_lb.grid(row=0, column=2)

# input entry
input_entry = ttk.Entry(Entrez_group,
                        textvariable=StringVar(),
                        width=30,
                        justify='center',
                        font=('Arial', 11,"bold"))
input_entry.grid(row=1, column=2, padx=10, pady=5)

# entrez buttton
Entrez_button = ttk.Button(Entrez_group,
                           text='Entrez',
                           command=get_results)
Entrez_button.grid(row=1, column=3, padx=10, pady=5)


root.mainloop()



Boiling Point Compound
